# Apply Operations To Groups In Pandas
> pandasのgroupbyについて外部サイトから学ぶ

- toc: true
- branch: master
- badges: true
- comments: false
- categories: [pandas, data analysis]

## 参考にしたサイト

* [GroupBy-fu: improvements in grouping and aggregating data in pandas - Wes McKinney](http://wesmckinney.com/blog/groupby-fu-improvements-in-grouping-and-aggregating-data-in-pandas/)
* [Working with DataFrames](http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/)
* [Apply Functions By Group In Pandas - Python](http://chrisalbon.com/python/pandas_apply_function_by_group.html)
* [Chris Albon - Data Science, Machine Learning, and Artificial Intelligence](http://chrisalbon.com/)
* [Pandas .groupby(), Lambda Functions, & Pivot Tables](https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/)
* [Summarising, Aggregating, and Grouping data in Python Pandas | Shane Lynn](http://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

## pandasで初期データを用意

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Create dataframe
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'],
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'],
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['regiment', 'company', 'name', 'preTestScore', 'postTestScore'])
df

,regiment,company,name,preTestScore,postTestScore
0,Nighthawks,1st,Miller,4,25
1,Nighthawks,1st,Jacobson,24,94
2,Nighthawks,2nd,Ali,31,57
3,Nighthawks,2nd,Milner,2,62
4,Dragoons,1st,Cooze,3,70
5,Dragoons,1st,Jacon,4,25
6,Dragoons,2nd,Ryaner,24,94
7,Dragoons,2nd,Sone,31,57
8,Scouts,1st,Sloan,2,62
9,Scouts,1st,Piger,3,70


## groupbyの使い方

In [3]:
# Create a groupby variable that groups preTestScores by regiment
groupby_regiment = df['preTestScore'].groupby(df['company'])
groupby_regiment

### groupsは、グループを作るときのインデックスの分類なので、df全体で処理しても同じ

In [9]:
groupby_regiment.groups

{'1st': Int64Index([0, 1, 4, 5, 8, 9], dtype='int64'),
 '2nd': Int64Index([2, 3, 6, 7, 10, 11], dtype='int64')}

In [7]:
df.groupby(df['company']).groups

{'1st': Int64Index([0, 1, 4, 5, 8, 9], dtype='int64'),
 '2nd': Int64Index([2, 3, 6, 7, 10, 11], dtype='int64')}

In [5]:
groupby_regiment.get_group("1st")

0     4
1    24
4     3
5     4
8     2
9     3
Name: preTestScore, dtype: int64

### 具体的に要素を確認するにはlist
gbの内容を視覚的に確認できる

In [4]:
list(groupby_regiment)

[('1st', 0     4
  1    24
  4     3
  5     4
  8     2
  9     3
  Name: preTestScore, dtype: int64), ('2nd', 2     31
  3      2
  6     24
  7     31
  10     2
  11     3
  Name: preTestScore, dtype: int64)]

In [10]:
groupby_regiment.describe()

company       
1st      count     6.000000
         mean      6.666667
         std       8.524475
         min       2.000000
         25%       3.000000
         50%       3.500000
         75%       4.000000
         max      24.000000
2nd      count     6.000000
         mean     15.500000
         std      14.652645
         min       2.000000
         25%       2.250000
         50%      13.500000
         75%      29.250000
         max      31.000000
Name: preTestScore, dtype: float64

In [14]:
groupby_regiment.mean()

company
1st     6.666667
2nd    15.500000
Name: preTestScore, dtype: float64

### groupbyで複数のカラムを設定すると、マルチインデックスになる

In [15]:
df['preTestScore'].groupby([df['regiment'], df['company']]).mean()

regiment    company
Dragoons    1st         3.5
            2nd        27.5
Nighthawks  1st        14.0
            2nd        16.5
Scouts      1st         2.5
            2nd         2.5
Name: preTestScore, dtype: float64

In [17]:
df['preTestScore'].groupby([df['regiment'], df['company']]).groups

{('Dragoons', '1st'): Int64Index([4, 5], dtype='int64'),
 ('Dragoons', '2nd'): Int64Index([6, 7], dtype='int64'),
 ('Nighthawks', '1st'): Int64Index([0, 1], dtype='int64'),
 ('Nighthawks', '2nd'): Int64Index([2, 3], dtype='int64'),
 ('Scouts', '1st'): Int64Index([8, 9], dtype='int64'),
 ('Scouts', '2nd'): Int64Index([10, 11], dtype='int64')}

df['preTestScore']のようにカラム指定からgroupbyするなら、groupby(arg)のargは、インデックスをわたす必要がある。カラム名は無効

In [31]:
try:
    df['preTestScore'].groupby(['regiment','company']).groups
except KeyError:
    print("エラー")

エラー


In [25]:
df.groupby(['regiment','company']).groups

{('Dragoons', '1st'): Int64Index([4, 5], dtype='int64'),
 ('Dragoons', '2nd'): Int64Index([6, 7], dtype='int64'),
 ('Nighthawks', '1st'): Int64Index([0, 1], dtype='int64'),
 ('Nighthawks', '2nd'): Int64Index([2, 3], dtype='int64'),
 ('Scouts', '1st'): Int64Index([8, 9], dtype='int64'),
 ('Scouts', '2nd'): Int64Index([10, 11], dtype='int64')}

In [27]:
df.groupby(['regiment','company']).mean()

preTestScore  postTestScore
regiment   company                             
Dragoons   1st               3.5           47.5
           2nd              27.5           75.5
Nighthawks 1st              14.0           59.5
           2nd              16.5           59.5
Scouts     1st               2.5           66.0
           2nd               2.5           66.0

In [36]:
df.groupby(['regiment','company']).mean().unstack()

preTestScore       postTestScore      
company             1st   2nd           1st   2nd
regiment                                         
Dragoons            3.5  27.5          47.5  75.5
Nighthawks         14.0  16.5          59.5  59.5
Scouts              2.5   2.5          66.0  66.0

In [37]:
df.groupby(['regiment', 'company']).size()

regiment    company
Dragoons    1st        2
            2nd        2
Nighthawks  1st        2
            2nd        2
Scouts      1st        2
            2nd        2
dtype: int64

### イテレータを使って、gbの内容を可視化

In [41]:
for name, group in df.groupby('regiment'):
    # print the name of the regiment
    print(name)
    # print the data of that regiment
    print(group)

Dragoons
   regiment company    name  preTestScore  postTestScore
4  Dragoons     1st   Cooze             3             70
5  Dragoons     1st   Jacon             4             25
6  Dragoons     2nd  Ryaner            24             94
7  Dragoons     2nd    Sone            31             57
Nighthawks
     regiment company      name  preTestScore  postTestScore
0  Nighthawks     1st    Miller             4             25
1  Nighthawks     1st  Jacobson            24             94
2  Nighthawks     2nd       Ali            31             57
3  Nighthawks     2nd    Milner             2             62
Scouts
   regiment company   name  preTestScore  postTestScore
8    Scouts     1st  Sloan             2             62
9    Scouts     1st  Piger             3             70
10   Scouts     2nd  Riani             2             62
11   Scouts     2nd    Ali             3             70


In [42]:
df.groupby('regiment').mean()

,preTestScore,postTestScore
regiment,,
Dragoons,15.50,61.5
Nighthawks,15.25,59.5
Scouts,2.50,66.0


In [47]:
df.groupby(['regiment', 'company']).mean().add_prefix('mean_').unstack()

mean_preTestScore       mean_postTestScore      
company                  1st   2nd                1st   2nd
regiment                                                   
Dragoons                 3.5  27.5               47.5  75.5
Nighthawks              14.0  16.5               59.5  59.5
Scouts                   2.5   2.5               66.0  66.0

### gb.apply(f(x))
gb.apply(f(x))を使う。
f(x)のxには、gbの要素のdfが代入される。

In [52]:
df['postTestScore'].groupby(df['regiment']).apply(lambda sdf: sdf.mean())

regiment
Dragoons      61.5
Nighthawks    59.5
Scouts        66.0
Name: postTestScore, dtype: float64

In [53]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

In [54]:
df['postTestScore'].groupby(df['regiment']).apply(get_stats)

regiment         
Dragoons    count     4.0
            max      94.0
            mean     61.5
            min      25.0
Nighthawks  count     4.0
            max      94.0
            mean     59.5
            min      25.0
Scouts      count     4.0
            max      70.0
            mean     66.0
            min      62.0
Name: postTestScore, dtype: float64

In [55]:
df['postTestScore'].groupby(df['regiment']).apply(get_stats).unstack()

,count,max,mean,min
regiment,,,,
Dragoons,4.0,94.0,61.5,25.0
Nighthawks,4.0,94.0,59.5,25.0
Scouts,4.0,70.0,66.0,62.0
